In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import os
from pathlib import Path

# ----------------------
# Create Results Directory Structure
# ----------------------
def create_results_directories():
    """Create directory structure for storing results"""
    directories = [
        'results',
        'results/cross_validation',
        'results/external_validation',
        'results/monte_carlo',
        'results/comparison',
        'results/train_predictions',
        'results/test_predictions'
    ]

    for directory in directories:
        os.makedirs(directory, exist_ok=True)

    print("Results directory structure created")

# ----------------------
# 1. Data Preparation and Preprocessing
# ----------------------
# Load data
merged_df = pd.read_csv('./Train_Test.csv')

# Calculate weighted sum
weights1 = {'D1': 10, 'D3': 9, 'D2': 9, 'D6': 13, 'D5': 10, 'D4': 14}
weights2 = {'D7-1': 5.90, 'D7-7': 3.54, 'D7-2': 5.90, 'D7-5': 4.33, 'D7-8': 3.13, 'D7-3': 3.15, 'D7-4': 5.90, 'D7-6': 3.15}
weighted_sum1 = sum(merged_df[feature] * weight for feature, weight in weights1.items())
weighted_sum2 = sum(merged_df[feature] * weight for feature, weight in weights2.items())
merged_df['target'] = weighted_sum1 + weighted_sum2

# Dataset splitting
X = merged_df.drop(['cas', 'target'], axis=1).values
y = merged_df['target'].values

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
# Standardization
scaler = StandardScaler()
X_train_val = scaler.fit_transform(X_train_val)
X_test = scaler.transform(X_test)

# ----------------------
# 2. Model Definitions
# ----------------------
class MLP(nn.Module):
    """Multi-Layer Perceptron with dropout for uncertainty estimation"""
    def __init__(self, input_dim, dropout_prob=0.001):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 32)
        self.fc6 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(dropout_prob)  # Single dropout layer instance

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.dropout(x)  # Apply dropout after intermediate hidden layers
        x = torch.relu(self.fc5(x))
        return self.fc6(x)  # No dropout before output layer

class SelfAttention(nn.Module):
    """Self-attention mechanism for capturing feature interactions"""
    def __init__(self, input_dim):
        super(SelfAttention, self).__init__()
        self.query = nn.Linear(input_dim, input_dim)
        self.key = nn.Linear(input_dim, input_dim)
        self.value = nn.Linear(input_dim, input_dim)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)
        attention_weights = self.softmax(torch.bmm(Q, K.transpose(1, 2)) / (x.shape[2] ** 0.5))
        return torch.bmm(attention_weights, V)

class MLP_Attention(nn.Module):
    """MLP with self-attention mechanism and dropout"""
    def __init__(self, input_dim, dropout_prob=0.01):  # Add dropout_prob parameter
        super(MLP_Attention, self).__init__()
        self.attention = SelfAttention(input_dim)
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 32)
        self.fc6 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(dropout_prob)  # Create dropout layer

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.attention(x)
        x = x.squeeze(1)

        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)

        x = torch.relu(self.fc3(x))
        x = self.dropout(x)

        x = torch.relu(self.fc4(x))
        x = self.dropout(x)

        x = torch.relu(self.fc5(x))
        return self.fc6(x)

# 1D-CNN Model
class CNNModel(nn.Module):
    """Convolutional Neural Network for tabular data"""
    def __init__(self, input_dim):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.bn1   = nn.BatchNorm1d(32)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.bn2   = nn.BatchNorm1d(64)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn3   = nn.BatchNorm1d(128)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.fc1   = nn.Linear(128 * (input_dim // 8), 64)
        self.dropout = nn.Dropout(0.001)
        self.fc2   = nn.Linear(64,1)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool3(torch.relu(self.bn3(self.conv3(x))))

        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        out = self.fc2(x)
        return out

class CNN_Attention(nn.Module):
    """1D-CNN with self-attention mechanism"""
    def __init__(self, input_dim):
        super(CNN_Attention, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32,
                               kernel_size=5, stride=1, padding=2)
        self.bn1   = nn.BatchNorm1d(32)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64,
                               kernel_size=5, stride=1, padding=2)
        self.bn2   = nn.BatchNorm1d(64)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128,
                               kernel_size=3, stride=1, padding=1)
        self.bn3   = nn.BatchNorm1d(128)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.attention = SelfAttention(128)

        self.fc_input_dim = (input_dim // 8) * 128

        self.fc1 = nn.Linear(self.fc_input_dim, 64)
        self.dropout = nn.Dropout(0.001)
        self.fc2 = nn.Linear(64, 32)
        self.fc3  = nn.Linear(32, 1)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool3(torch.relu(self.bn3(self.conv3(x))))
        x = x.transpose(1, 2)
        x = self.attention(x)
        x = x.reshape(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        out = self.fc3(x)
        return out

class AttentionModel(nn.Module):
    """Attention-based model for regression"""
    def __init__(self, input_dim, dropout_rate=0.001):
        super(AttentionModel, self).__init__()
        self.attention = SelfAttention(input_dim)
        self.fc1 = nn.Linear(input_dim, 256)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(256, 1)

    def forward(self, x):
        x = self.attention(x.unsqueeze(1)).squeeze(1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

# ----------------------
# Device Selection
# ----------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ----------------------
# 3. Load External Validation Set
# ----------------------
def load_external_validation_set(file_path='./External_Validation.csv'):
    """Load and preprocess external validation dataset"""
    try:
        ext_df = pd.read_csv(file_path)
        print(f"Successfully loaded external validation set: {file_path}")
        print(f"Shape of external validation set: {ext_df.shape}")

        # Calculate weighted sum (same method as training data)
        weighted_sum1 = sum(ext_df[feature] * weight for feature, weight in weights1.items())
        weighted_sum2 = sum(ext_df[feature] * weight for feature, weight in weights2.items())
        ext_df['target'] = weighted_sum1 + weighted_sum2

        # Extract features and target
        casNumber = ext_df['cas']
        X_ext = ext_df.drop(['cas', 'target'], axis=1).values
        y_ext = ext_df['target'].values

        # Standardize using the same scaler as training data
        X_ext_scaled = scaler.transform(X_ext)

        return X_ext_scaled, y_ext, casNumber
    except Exception as e:
        print(f"Error loading external validation set: {e}")
        return None, None, None

# ----------------------
# 4. Monte Carlo Dropout Implementation
# ----------------------
def monte_carlo_prediction(model, X, n_samples=1000, dropout_prob=0.1):
    """
    Perform prediction using Monte Carlo dropout for uncertainty estimation

    Args:
    - model: Trained PyTorch model
    - X: Input features (numpy array)
    - n_samples: Number of Monte Carlo samples
    - dropout_prob: Dropout probability

    Returns:
    - mean_pred: Mean prediction
    - std_pred: Standard deviation
    - ci_lower: 95% confidence interval lower bound
    - ci_upper: 95% confidence interval upper bound
    - all_preds: All predictions
    """
    # Set model to evaluation mode but keep dropout enabled
    model.eval()

    # Set dropout probability for all dropout layers in the model
    for m in model.modules():
        if isinstance(m, nn.Dropout):
            m.p = dropout_prob
            m.train()  # Enable dropout even in eval mode

    # Convert to PyTorch tensor
    X_tensor = torch.FloatTensor(X).to(device)

    # Store all predictions
    all_preds = []

    # Perform multiple predictions
    with torch.no_grad():
        for _ in range(n_samples):
            preds = model(X_tensor).cpu().numpy().flatten()
            all_preds.append(preds)

    # Convert predictions to numpy array [n_samples, n_samples]
    all_preds = np.array(all_preds)

    # Calculate statistics
    mean_pred = np.mean(all_preds, axis=0)
    std_pred = np.std(all_preds, axis=0)

    # Calculate 95% confidence interval
    ci_lower = np.percentile(all_preds, 2.5, axis=0)
    ci_upper = np.percentile(all_preds, 97.5, axis=0)

    return mean_pred, std_pred, ci_lower, ci_upper, all_preds

# ----------------------
# 5. Modified Cross-Validation Framework
# ----------------------
def kfold_validate(model_class, X_train_val, y_train_val, n_splits=10, epochs=1500, patience=150, lr=0.001, monte_carlo=True):
    """
    Evaluate model performance using cross-validation, supporting 5-fold and 10-fold validation,
    and save results as CSV

    Args:
    - model_class: Model class
    - X_train_val: Training and validation features
    - y_train_val: Training and validation targets
    - n_splits: Number of folds (5 or 10)
    - epochs: Number of training epochs
    - patience: Early stopping patience
    - lr: Learning rate
    - monte_carlo: Whether to use Monte Carlo dropout for uncertainty estimation

    Returns:
    - metrics: Dictionary of evaluation metrics
    """
    kf = KFold(n_splits=n_splits)
    metrics = {'r2': [], 'mae': [], 'mse': []}

    if monte_carlo:
        metrics.update({'std': [], 'ci_width': []})

    print(f"\n{'='*90}")
    print(f"Validating Model: {model_class.__name__} with {n_splits}-fold Cross Validation")
    print(f"{'='*90}")

    # Prepare CSV data
    csv_data = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_val)):
        # Data standardization (independent for each fold)
        print(f"\n{'='*45}")
        print(f"Fold: {fold+1}/{n_splits}")
        print(f"{'='*45}")
        X_train, X_val = X_train_val[train_idx], X_train_val[val_idx]
        y_train, y_val = y_train_val[train_idx], y_train_val[val_idx]

        fold_scaler = StandardScaler()
        X_train = fold_scaler.fit_transform(X_train)
        X_val = fold_scaler.transform(X_val)

        # Convert to tensors and move to GPU
        X_train_t = torch.FloatTensor(X_train).to(device)
        y_train_t = torch.FloatTensor(y_train).unsqueeze(1).to(device)
        X_val_t = torch.FloatTensor(X_val).to(device)
        y_val_t = torch.FloatTensor(y_val).unsqueeze(1).to(device)

        # Initialize model and move to GPU
        model = model_class(X_train.shape[1]).to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr)
        criterion = nn.MSELoss()

        best_loss = float('inf')
        wait = 0
        for epoch in range(epochs):
            model.train()
            optimizer.zero_grad()
            outputs = model(X_train_t)
            loss = criterion(outputs, y_train_t)
            loss.backward()
            optimizer.step()

            # Validation evaluation
            model.eval()
            with torch.no_grad():
                val_loss = criterion(model(X_val_t), y_val_t)

            # Print every 50 epochs
            if (epoch + 1) % 50 == 0 or epoch == 0:
                print(f"[Fold {fold+1}/{n_splits}] Epoch {epoch+1:04d}/{epochs:04d} | "
                      f"Train Loss: {loss.item():.4f} | "
                      f"Val Loss: {val_loss.item():.4f} | "
                      f"Wait: {wait}/{patience}")

            # Early stopping mechanism
            if val_loss < best_loss:
                best_loss = val_loss.item()
                wait = 0
            else:
                wait += 1
                if wait >= patience:
                    break

        # Record metrics (move results back to CPU)
        with torch.no_grad():
            preds = model(X_val_t).cpu().numpy().flatten()

        # Basic metrics
        r2 = r2_score(y_val, preds)
        mae = mean_absolute_error(y_val, preds)
        mse = mean_squared_error(y_val, preds)

        metrics['r2'].append(r2)
        metrics['mae'].append(mae)
        metrics['mse'].append(mse)

        # Prepare CSV row data
        fold_data = {
            'fold_id': fold + 1,
            'r2': r2,
            'mae': mae,
            'mse': mse
        }

        # Monte Carlo uncertainty estimation
        if monte_carlo:
            mean_pred, std_pred, ci_lower, ci_upper, _ = monte_carlo_prediction(model, X_val)
            mean_std = np.mean(std_pred)
            mean_ci_width = np.mean(ci_upper - ci_lower)

            metrics['std'].append(mean_std)
            metrics['ci_width'].append(mean_ci_width)

            # Add Monte Carlo statistics to CSV row
            fold_data.update({
                'mean_std': mean_std,
                'mean_ci_width': mean_ci_width,
                'mean_ci_lower': np.mean(ci_lower),
                'mean_ci_upper': np.mean(ci_upper)
            })

            print(f"\n📊 Monte Carlo Uncertainty Estimation (Fold {fold+1}/{n_splits}):")
            print(f"Average Standard Deviation: {mean_std:.4f}")
            print(f"Average 95% Confidence Interval Width: {mean_ci_width:.4f}")
            print(f"Mean CI Lower Bound: {np.mean(ci_lower):.4f}")
            print(f"Mean CI Upper Bound: {np.mean(ci_upper):.4f}")

        # Add to CSV data
        csv_data.append(fold_data)

    # Calculate and print cross-validation results
    print(f"\n🔍 {n_splits}-Fold Cross-Validation Results for {model_class.__name__}:")
    print(f"R²: {np.mean(metrics['r2']):.4f} ± {np.std(metrics['r2']):.4f}")
    print(f"MAE: {np.mean(metrics['mae']):.4f} ± {np.std(metrics['mae']):.4f}")
    print(f"MSE: {np.mean(metrics['mse']):.4f} ± {np.std(metrics['mse']):.4f}")

    if monte_carlo:
        print(f"Average Standard Deviation: {np.mean(metrics['std']):.4f}")
        print(f"Average 95% Confidence Interval Width: {np.mean(metrics['ci_width']):.4f}")

    # Add summary rows
    summary_row = {
        'fold_id': 'mean',
        'r2': np.mean(metrics['r2']),
        'mae': np.mean(metrics['mae']),
        'mse': np.mean(metrics['mse'])
    }

    std_row = {
        'fold_id': 'std',
        'r2': np.std(metrics['r2']),
        'mae': np.std(metrics['mae']),
        'mse': np.std(metrics['mse'])
    }

    if monte_carlo:
        summary_row.update({
            'mean_std': np.mean(metrics['std']),
            'mean_ci_width': np.mean(metrics['ci_width'])
        })
        std_row.update({
            'mean_std': np.std(metrics['std']),
            'mean_ci_width': np.std(metrics['ci_width'])
        })

    csv_data.append(summary_row)
    csv_data.append(std_row)

    # Save as CSV
    df = pd.DataFrame(csv_data)
    csv_path = f"results/cross_validation/{model_class.__name__}_{n_splits}fold_cv_results.csv"
    df.to_csv(csv_path, index=False)
    print(f"Cross-validation results saved to: {csv_path}")

    return metrics

# ----------------------
# 6. Modified Training Framework
# ----------------------
def train_final_model(model_class, X_train_val, y_train_val, lr=0.001):
    """Train the final model using all training data"""
    # Standardization
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_train_val)

    # Convert to tensors and move to GPU
    X_t = torch.FloatTensor(X_scaled).to(device)
    y_t = torch.FloatTensor(y_train_val).unsqueeze(1).to(device)

    # Initialize model and move to GPU
    model = model_class(X_scaled.shape[1]).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    # Training (use all data, no early stopping)
    for epoch in range(1500):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_t)
        loss = criterion(outputs, y_t)
        loss.backward()
        optimizer.step()

    # Save the model (move to CPU before saving)
    model.to("cpu")  # Move to CPU before saving
    torch.save({
        'model_state': model.state_dict(),
        'scaler': scaler  # Save the scaler
    }, f'FINAL_{model_class.__name__}.pth')

    model.eval()  # Set to evaluation mode
    # Move the model to the same device as test data
    model = model.to(device)

    X_test_scaled = scaler.transform(X_test)  # Assume X_test is a global variable
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)

    with torch.no_grad():
        # Move training data to the same device as the model
        X_scaled_tensor = torch.FloatTensor(X_scaled).to(device)
        y_pred_train = model(X_scaled_tensor).cpu().numpy().flatten()  # Training set predictions
        y_pred_test = model(X_test_tensor).cpu().numpy().flatten()  # Test set predictions

    # Save training set results (optional)
    train_results = pd.DataFrame({
        'actual_value': y_train_val,
        'predicted_value': y_pred_train
    })
    train_csv_path = f"results/train_predictions/{model_class.__name__}_train_results.csv"
    os.makedirs(os.path.dirname(train_csv_path), exist_ok=True)
    train_results.to_csv(train_csv_path, index=False)
    print(f"Training set predictions saved to: {train_csv_path}")

    # Save test set results
    test_results = pd.DataFrame({
        'actual_value': y_test,
        'predicted_value': y_pred_test
    })
    test_csv_path = f"results/test_predictions/{model_class.__name__}_test_results.csv"
    os.makedirs(os.path.dirname(test_csv_path), exist_ok=True)
    test_results.to_csv(test_csv_path, index=False)
    print(f"Test set predictions saved to: {test_csv_path}")

    return model.to(device)  # Return moved back to GPU

# ----------------------
# 7. External Validation Set Evaluation
# ----------------------
def evaluate_on_external_set(model, X_ext, y_ext, model_name, casNumber, monte_carlo=True):
    """
    Evaluate model performance on external validation set and save results as CSV

    Args:
    - model: Trained PyTorch model
    - X_ext: External validation features
    - y_ext: External validation targets
    - model_name: Model name
    - monte_carlo: Whether to use Monte Carlo dropout for uncertainty estimation

    Returns:
    - metrics: Dictionary of evaluation metrics
    """
    model.eval()
    X_tensor = torch.FloatTensor(X_ext).to(device)

    # Basic prediction
    with torch.no_grad():
        preds = model(X_tensor).cpu().numpy().flatten()

    # Calculate basic metrics
    r2 = r2_score(y_ext, preds)
    mae = mean_absolute_error(y_ext, preds)
    mse = mean_squared_error(y_ext, preds)

    metrics = {
        'r2': r2,
        'mae': mae,
        'mse': mse
    }

    print(f"\n📋 External Validation Set Results:")
    print(f"R²: {r2:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")

    # Prepare CSV data
    csv_data = [{
        'r2': r2,
        'mae': mae,
        'mse': mse
    }]

    # Monte Carlo uncertainty estimation
    if monte_carlo:
        mean_pred, std_pred, ci_lower, ci_upper, all_preds = monte_carlo_prediction(model, X_ext)

        mean_std = np.mean(std_pred)
        mean_ci_width = np.mean(ci_upper - ci_lower)

        metrics.update({
            'mean_pred': mean_pred,
            'std_pred': std_pred,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper,
            'mean_std': mean_std,
            'mean_ci_width': mean_ci_width
        })

        # Update CSV data
        csv_data[0].update({
            'mean_std': mean_std,
            'mean_ci_width': mean_ci_width,
            'mean_ci_lower': np.mean(ci_lower),
            'mean_ci_upper': np.mean(ci_upper)
        })

        print(f"\n📊 Monte Carlo Uncertainty Estimation on External Set:")
        print(f"Average Standard Deviation: {mean_std:.4f}")
        print(f"Average 95% Confidence Interval Width: {mean_ci_width:.4f}")
        print(f"Mean CI Lower Bound: {np.mean(ci_lower):.4f}")
        print(f"Mean CI Upper Bound: {np.mean(ci_upper):.4f}")

        # Save detailed Monte Carlo statistics
        mc_data = []
        for i in range(len(y_ext)):
            mc_data.append({
                'sample_id': i,
                'true_value': y_ext[i],
                'mean_pred': mean_pred[i],
                'std': std_pred[i],
                'ci_lower': ci_lower[i],
                'ci_upper': ci_upper[i]
            })

        mc_df = pd.DataFrame(mc_data)
        mc_csv_path = f"results/monte_carlo/{model_name}_monte_carlo_stats.csv"
        mc_df.to_csv(mc_csv_path, index=False)
        print(f"Monte Carlo statistics saved to: {mc_csv_path}")

    # Save external validation results
    ext_df = pd.DataFrame(csv_data)
    ext_csv_path = f"results/external_validation/{model_name}_external_validation.csv"
    ext_df.to_csv(ext_csv_path, index=False)
    print(f"External validation results saved to: {ext_csv_path}")

    # Save detailed actual vs predicted values
    ext_detail_data = pd.DataFrame({
        'cas': casNumber,
        'actual_value': y_ext,
        'predicted_value': preds
    })

    # Add Monte Carlo prediction intervals (optional)
    if monte_carlo:
        ext_detail_data['mean_pred'] = metrics['mean_pred']
        ext_detail_data['ci_lower'] = metrics['ci_lower']
        ext_detail_data['ci_upper'] = metrics['ci_upper']

    ext_detail_csv_path = f"results/external_validation/{model_name}_external_detail.csv"
    ext_detail_data.to_csv(ext_detail_csv_path, index=False)
    print(f"Detailed external validation results (actual vs predicted) saved to: {ext_detail_csv_path}")

    return metrics

# ----------------------
# 8. Main Function: Run All Models
# ----------------------
def run_all_models():
    """Run training, validation, and evaluation for all models, and save results as CSV"""
    # Create results directories
    create_results_directories()

    # Load external validation set
    X_ext, y_ext, casNumber = load_external_validation_set()

    if X_ext is None or y_ext is None:
        print("Unable to load external validation set, skipping external validation evaluation")

    # List of models
    models = [MLP, MLP_Attention, CNNModel, CNN_Attention, AttentionModel]
    model_names = ["MLP", "MLP_Attention", "CNN", "CNN_Attention", "Attention"]

    # Results storage
    results = {
        '5-fold': {},
        '10-fold': {},
        'external': {}
    }

    # Prepare comparison data
    comparison_data = []

    # Perform 5-fold and 10-fold validation for each model
    for model_class, model_name in zip(models, model_names):
        print(f"\n{'#'*100}")
        print(f"Processing Model: {model_name}")
        print(f"{'#'*100}")

        # 5-fold cross-validation
        print("\n🔄 Running 5-fold Cross Validation...")
        results['5-fold'][model_name] = kfold_validate(model_class, X_train_val, y_train_val, n_splits=5)

        # 10-fold cross-validation
        print("\n🔄 Running 10-fold Cross Validation...")
        results['10-fold'][model_name] = kfold_validate(model_class, X_train_val, y_train_val, n_splits=10)

        # Train final model
        print("\n🏆 Training Final Model...")
        final_model = train_final_model(model_class, X_train_val, y_train_val)

        # Evaluate on external validation set (if available)
        if X_ext is not None and y_ext is not None:
            print("\n🔍 Evaluating on External Validation Set...")
            results['external'][model_name] = evaluate_on_external_set(final_model, X_ext, y_ext, model_name, casNumber)

        # Add to comparison data
        # 5-fold results
        fold5_row = {
            'model_name': model_name,
            'fold_type': '5-fold',
            'r2_mean': np.mean(results['5-fold'][model_name]['r2']),
            'r2_std': np.std(results['5-fold'][model_name]['r2']),
            'mae_mean': np.mean(results['5-fold'][model_name]['mae']),
            'mae_std': np.std(results['5-fold'][model_name]['mae']),
            'mse_mean': np.mean(results['5-fold'][model_name]['mse']),
            'mse_std': np.std(results['5-fold'][model_name]['mse'])
        }

        # 10-fold results
        fold10_row = {
            'model_name': model_name,
            'fold_type': '10-fold',
            'r2_mean': np.mean(results['10-fold'][model_name]['r2']),
            'r2_std': np.std(results['10-fold'][model_name]['r2']),
            'mae_mean': np.mean(results['10-fold'][model_name]['mae']),
            'mae_std': np.std(results['10-fold'][model_name]['mae']),
            'mse_mean': np.mean(results['10-fold'][model_name]['mse']),
            'mse_std': np.std(results['10-fold'][model_name]['mse'])
        }

        # Add Monte Carlo statistics (if available)
        if 'std' in results['5-fold'][model_name]:
            fold5_row.update({
                'mc_std': np.mean(results['5-fold'][model_name]['std']),
                'mc_ci_width': np.mean(results['5-fold'][model_name]['ci_width'])
            })
            fold10_row.update({
                'mc_std': np.mean(results['10-fold'][model_name]['std']),
                'mc_ci_width': np.mean(results['10-fold'][model_name]['ci_width'])
            })

        comparison_data.append(fold5_row)
        comparison_data.append(fold10_row)

    # Save comparison results
    comparison_df = pd.DataFrame(comparison_data)
    comparison_csv_path = "results/comparison/model_comparison.csv"
    comparison_df.to_csv(comparison_csv_path, index=False)
    print(f"Model comparison results saved to: {comparison_csv_path}")

    # Print comparison results
    print("\n📊 Comparison of 5-fold and 10-fold Cross Validation Results:")
    for model_name in model_names:
        print(f"\n{'-'*50}")
        print(f"Model: {model_name}")
        print(f"{'-'*50}")

        print(f"5-fold R²: {np.mean(results['5-fold'][model_name]['r2']):.4f} ± {np.std(results['5-fold'][model_name]['r2']):.4f}")
        print(f"10-fold R²: {np.mean(results['10-fold'][model_name]['r2']):.4f} ± {np.std(results['10-fold'][model_name]['r2']):.4f}")

        print(f"5-fold MAE: {np.mean(results['5-fold'][model_name]['mae']):.4f} ± {np.std(results['5-fold'][model_name]['mae']):.4f}")
        print(f"10-fold MAE: {np.mean(results['10-fold'][model_name]['mae']):.4f} ± {np.std(results['10-fold'][model_name]['mae']):.4f}")

        print(f"5-fold MSE: {np.mean(results['5-fold'][model_name]['mse']):.4f} ± {np.std(results['5-fold'][model_name]['mse']):.4f}")
        print(f"10-fold MSE: {np.mean(results['10-fold'][model_name]['mse']):.4f} ± {np.std(results['10-fold'][model_name]['mse']):.4f}")

        if 'std' in results['5-fold'][model_name]:
            print(f"5-fold Average Standard Deviation: {np.mean(results['5-fold'][model_name]['std']):.4f}")
            print(f"10-fold Average Standard Deviation: {np.mean(results['10-fold'][model_name]['std']):.4f}")

            print(f"5-fold Average 95% Confidence Interval Width: {np.mean(results['5-fold'][model_name]['ci_width']):.4f}")
            print(f"10-fold Average 95% Confidence Interval Width: {np.mean(results['10-fold'][model_name]['ci_width']):.4f}")

    return results

# If this script is run directly, execute training and evaluation for all models
if __name__ == "__main__":
    run_all_models()